# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="C:/Users/Marcin i Ewa/Udacity/deep-reinforcement-learning/p1_navigation/Banana_Windows_x86_64/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
print(env_info)
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
from agent import Agent
import torch
episode_number = 1500
eps_start=1.0
eps_end=0.01
eps_decay=0.995
max_steps = 500
agent = Agent(action_number = brain.vector_action_space_size, state_size = len(env_info.vector_observations[0]))

In [ ]:
def preprocess():
    print("starting preprocessing")
    end_preprocessing = False
    while not end_preprocessing:
        print(len(agent.memoryBuffer))
        env_info = env.reset(train_mode=False)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        while True:
            action = np.random.choice(brain.vector_action_space_size)            # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.memoryBuffer.add(state, action, reward, next_state, done)
            state = next_state                             # roll over the state to next time step
            if len(agent.memoryBuffer) == 4999:
                end_preprocessing = True
            if done or len(agent.memoryBuffer) >=4999:
                break
    print("preprocessing done", len(agent.memoryBuffer))

In [ ]:
episode_score_buffer = []
eps = eps_start
preprocess()
for episode in range(episode_number):
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.choose_action(state, eps).astype(int)            # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        agent.step(state, action, reward, next_state, done)
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break
    print(episode, " Score: {}".format(score))
    if len(episode_score_buffer) > 100:
        print("      ", episode, "  sum of last 100 :", np.sum(episode_score_buffer[-100:]))
    eps = eps*eps_decay
    if eps < eps_end:
        eps = eps_end
    episode_score_buffer.append(score)
    if (len(episode_score_buffer) > 100) and (np.sum(episode_score_buffer[-100:])/100 >= 13):
        print("End in episode ", episode)
        torch.save(agent.localNetwork.state_dict(), 'checkpoint.pth')
        break



starting preprocessing
0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
preprocessing done 4999
0  Score: -1.0
1  Score: -1.0
2  Score: -2.0
3  Score: -1.0
4  Score: 0.0
5  Score: 0.0
6  Score: 0.0
7  Score: -1.0
8  Score: 0.0
9  Score: 0.0
10  Score: 1.0
11  Score: 0.0
12  Score: -1.0
13  Score: 1.0
14  Score: 0.0
15  Score: -3.0
16  Score: 1.0
17  Score: 0.0
18  Score: 1.0
19  Score: -1.0
20  Score: -2.0
21  Score: 1.0
22  Score: 0.0
23  Score: -2.0
24  Score: 1.0
25  Score: 1.0
26  Score: 0.0
27  Score: 0.0
28  Score: 0.0
29  Score: -3.0
30  Score: 0.0
31  Score: 0.0
32  Score: -1.0
33  Score: 2.0
34  Score: 0.0
35  Score: 1.0
36  Score: 0.0
37  Score: 0.0
38  Score: 0.0
39  Score: -1.0
40  Score: 1.0
41  Score: 0.0
42  Score: 0.0
43  Score: 2.0
44  Score: 2.0
45  Score: 2.0
46  Score: 3.0
47  Score: 0.0
48  Score: 0.0
49  Score: 3.0
50  Score: 1.0
51  Score: 0.0
52  Score: 0.0
53  Score: -1.0
54  Score: -1.0
55  Score: 2.0
56  Score: 0.0
57  Score: 1.0

When finished, you can close the environment.

In [ ]:
agent.localNetwork.load_state_dict(torch.load('checkpoint200.pth'))
score =0
while True:
    action = agent.choose_action(state, 0.0).astype(int)            # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    print(score)
    if done:                                       # exit loop if episode finished
        break

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(episode_score_buffer)), episode_score_buffer)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]
score = 0
